## A3c : Fitting Tobit Model
                    Aiman Magi
                    V01106546

In [17]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import norm
from statsmodels.base.model import GenericLikelihoodModel

In [19]:
# Load the dataset
df = pd.read_csv("/Users/aimanmagi/Desktop/R/NSSO68.csv")
df.head()

/var/folders/k7/s0m9tqj95w31nsksdy12g29w0000gn/T/ipykernel_35876/795237538.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/aimanmagi/Desktop/R/NSSO68.csv")


,slno,grp,Round_Centre,FSU_number,Round,Schedule_Number,Sample,Sector,state,State_Region,...,pickle_v,sauce_jam_v,Othrprocessed_v,Beveragestotal_v,foodtotal_v,foodtotal_q,state_1,Region,fruits_df_tt_v,fv_tot
0,1,40999999999999992652495293775872.0,1,41000,68,10,1,2,24,242,...,0.0,0.0,0.0,0.000000,1141.492400,30.942394,GUJ,2,12.000000,154.18
1,2,40999999999999992652495293775872.0,1,41000,68,10,1,2,24,242,...,0.0,0.0,0.0,17.500000,1244.553500,29.286153,GUJ,2,333.000000,484.95
2,3,40999999999999992652495293775872.0,1,41000,68,10,1,2,24,242,...,0.0,0.0,0.0,0.000000,1050.315400,31.527046,GUJ,2,35.000000,214.84
3,4,40999999999999992652495293775872.0,1,41000,68,10,1,2,24,242,...,0.0,0.0,0.0,33.333333,1142.591667,27.834607,GUJ,2,168.333333,302.30
4,5,40999999999999992652495293775872.0,1,41000,68,10,1,2,24,242,...,0.0,0.0,0.0,75.000000,945.249500,27.600713,GUJ,2,15.000000,148.00


In [3]:
# Data preprocessing
# Check for missing values
print(df.isnull().sum())

slno              0
grp               0
Round_Centre      0
FSU_number        0
Round             0
                 ..
foodtotal_q       0
state_1           0
Region            0
fruits_df_tt_v    0
fv_tot            0
Length: 384, dtype: int64


In [4]:
# Choose dependent and independent variables
# Assuming 'MPCE_URP' (Monthly Per Capita Expenditure using Uniform Reference Period) is the dependent variable
# and 'hhdsz' (household size) as one of the independent variables
# You can choose other variables based on your analysis

dependent_var = 'MPCE_URP'
independent_vars = ['hhdsz', 'Age', 'Sex', 'Education']  # Add more variables as needed

In [5]:
# Prepare the data for regression
X = df[independent_vars]
y = df[dependent_var]

In [6]:
X.head()

,hhdsz,Age,Sex,Education
0,5,50,1,8.0
1,2,40,2,12.0
2,5,45,1,7.0
3,3,75,1,6.0
4,4,30,1,7.0


In [7]:
y.head()

0    3304.80
1    7613.00
2    3461.40
3    3339.00
4    2604.25
Name: MPCE_URP, dtype: float64

In [8]:
# Handle missing values: Option 1 - Remove rows with missing values
# X = X.dropna()
# y = y.loc[X.index]

In [9]:
# Add a constant to the independent variables matrix
X = sm.add_constant(X)

In [10]:
# Check for and remove any inf values
X.replace([np.inf, -np.inf], np.nan, inplace=True)
X.dropna(inplace=True)

In [11]:
# Check and ensure no NaN values remain
if X.isnull().any().any():
    raise ValueError("There are still NaN values in the independent variables matrix.")

In [12]:
# Ensure dependent variable matches the cleaned X
y = y.loc[X.index]

In [16]:
# Define a custom Tobit model class
class TobitModel(GenericLikelihoodModel):
    def __init__(self, endog, exog, left=None, right=None, **kwargs):
        self.left = left
        self.right = right
        super(TobitModel, self).__init__(endog, exog, **kwargs)

    def nloglikeobs(self, params):
        exog = self.exog
        endog = self.endog
        left = self.left
        right = self.right

        beta = params[:-1]
        sigma = params[-1]
        XB = np.dot(exog, beta)
        
        ll = np.zeros(len(endog))
        
        if left is not None:
            cdf_left = (endog <= left).astype(int)
            ll += cdf_left * np.log(1 - norm.cdf((left - XB) / sigma))
        
        if right is not None:
            cdf_right = (endog >= right).astype(int)
            ll += cdf_right * np.log(norm.cdf((right - XB) / sigma))
        
        uncensored = np.ones(len(endog), dtype=bool)
        if left is not None:
            uncensored &= (endog > left)
        if right is not None:
            uncensored &= (endog < right)
        
        ll[uncensored] = (norm.logpdf((endog[uncensored] - XB[uncensored]) / sigma) - np.log(sigma))

        return -ll

    def fit(self, start_params=None, maxiter=10000, maxfun=5000, **kwargs):
        if start_params is None:
            start_params = np.append(np.zeros(self.exog.shape[1]), 1)
        return super(TobitModel, self).fit(start_params=start_params, maxiter=maxiter, maxfun=maxfun, **kwargs)


In [17]:
# Set left censoring at 0 (lower bound)
left_censoring = 0

In [18]:
# Fit the Tobit model
model = TobitModel(y, X, left=left_censoring)
results = model.fit()

Optimization terminated successfully.
         Current function value: 9.750560
         Iterations: 716
         Function evaluations: 1151


C:\Users\gauri\anaconda3\Lib\site-packages\statsmodels\base\model.py:2742: UserWarning: df_model + k_constant + k_extra differs from k_params
  warnings.warn("df_model + k_constant + k_extra "
C:\Users\gauri\anaconda3\Lib\site-packages\statsmodels\base\model.py:2746: UserWarning: df_resid differs from nobs - k_params
  warnings.warn("df_resid differs from nobs - k_params")


In [19]:
# Print the summary of the regression results
print(results.summary())

                              TobitModel Results                              
Dep. Variable:               MPCE_URP   Log-Likelihood:            -9.9119e+05
Model:                     TobitModel   AIC:                         1.982e+06
Method:            Maximum Likelihood   BIC:                         1.982e+06
Date:                Sun, 30 Jun 2024                                         
Time:                        09:13:00                                         
No. Observations:              101655                                         
Df Residuals:                  101650                                         
Df Model:                           4                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -19.3490     74.697     -0.259      0.796    -165.752     127.055
hhdsz         25.5786      6.143      4.164      0.0